In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from keras.applications import VGG16
from keras import Sequential
from keras import regularizers
from keras.layers import Dense,Reshape,Flatten,Dropout,GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D ,Input
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as k
from keras import optimizers
from keras.models import Model, load_model

import os
from keras.utils import np_utils

Using TensorFlow backend.


In [ ]:
train_data = pd.read_csv('/kaggle/input/aptos2019-blindness-detection/train.csv')
tset_data = pd.read_csv('/kaggle/input/aptos2019-blindness-detection/test.csv')
train_data.head()

In [ ]:
def crop_image_from_gray(img, tol=7):
    """
    Applies masks to the orignal image and 
    returns the a preprocessed image with 
    3 channels
    """
    # If for some reason we only have two channels
    if img.ndim == 2:
        mask = img > tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    # If we have a normal RGB images
    elif img.ndim == 3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img > tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
            img = np.stack([img1,img2,img3],axis=-1)
        return img

def preprocess_image(path, sigmaX=10):
    """
    The whole preprocessing pipeline:
    1. Read in image
    2. Apply masks
    3. Resize image to desired size
    4. Add Gaussian noise to increase Robustness
    """
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = crop_image_from_gray(image)
    image = cv2.resize(image, (224, 224))
    image = cv2.addWeighted (image,4, cv2.GaussianBlur(image, (0,0) ,sigmaX), -4, 128)
    return image


N = train_data.shape[0]
x = np.empty((N,224,224,3),dtype=np.uint8)
for index,image_id in enumerate(tqdm(train_data['id_code'])):
    x[index,:,:,:]  = preprocess_image('/kaggle/input/aptos2019-blindness-detection/train_images/'+image_id+'.png')

In [ ]:
y = np_utils.to_categorical(train_data.diagnosis)
y.shape

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.15,shuffle=False,random_state=False)



data_gen  = ImageDataGenerator(rotation_range=15,
                     rescale=1./255,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   shear_range=0.01,
                                   zoom_range=[0.9, 1.25],
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   fill_mode='reflect',
                                   data_format='channels_last',
                                   brightness_range=[0.5, 1.5])

train_datagen = data_gen.flow(x_train,y_train,batch_size=32)
valid_datagen = data_gen.flow(x_test,y_test,batch_size=32)

In [ ]:
vgg = VGG16(weights=None,input_shape=(224,224,3),include_top=False,classes=5)
#vgg.load_weights('/kaggle/input/vgg16/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5')

model = Sequential()
for layer in vgg.layers:
    model.add(layer)
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(540,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.45))
model.add(Dense(5,activation='softmax'))
    
callback = ModelCheckpoint("best_weights.h5", monitor='acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto',
                             period=1)

early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto')
# Reducing the Learning Rate if result is not improving. 
reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.1, mode='auto',
                              verbose=1)

In [ ]:
model.compile(optimizer=optimizers.Adam(1e-4),
              loss="categorical_crossentropy", metrics=["accuracy"])
model.fit_generator(generator=train_datagen,
                    steps_per_epoch=98,
                    validation_data=valid_datagen,
                    validation_steps=18,
                    epochs=30,
                    callbacks = [callback,early_stop,reduce_lr])

In [ ]:
model.load_weights('best_weights.h5')
test_csv = pd.read_csv("/kaggle/input/aptos2019-blindness-detection/test.csv")
predicted_csv = pd.DataFrame(columns=["id_code", "diagnosis"])

N = test_csv.shape[0]
x_test = np.empty((N,224,224,3),dtype=np.uint8)
for index,image_id in enumerate(tqdm(test_csv['id_code'])):
    x_test[index,:,:,:]  = preprocess_image('/kaggle/input/aptos2019-blindness-detection/test_images/'+image_id+'.png')
    img = x_test[index]
    img = np.expand_dims(img, 0)
    prediction = int(np.argmax(model.predict(img)))
    predicted_csv = predicted_csv.append(
        {'id_code':image_id ,"diagnosis": prediction}, ignore_index=True)

with open("submission.csv", "w") as f:
    f.write(predicted_csv.to_csv(index=False))